In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from transformers import BitsAndBytesConfig
import torch

model_id = "huggyllama/llama-7b"
peft_model_id = './training/qlora-tasks-7b-v1.9.1/checkpoint-6000/adapter_model'

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    'huggyllama/llama-7b',
    load_in_4bit=True,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    ),
)
model = PeftModel.from_pretrained(model, peft_model_id)

tokenizer = AutoTokenizer.from_pretrained(model_id)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/boa/.conda/envs/ai/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/boa/.conda/envs/ai/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/boa/.conda/envs/ai/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/boa/.conda/envs/ai/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/boa/.conda/envs/ai/lib/libcudart.so'), PosixPath('/home/boa/.conda/envs/ai/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from datasets import concatenate_datasets, load_from_disk, DatasetDict
combined_tasks = load_from_disk('datasets/tasks/combined_task_dataset_io_notags/')

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


In [3]:
combined_tasks['test']['output'][0]

' egyéb\n'

In [4]:
combined_tasks = combined_tasks.filter(lambda data: data['input'].startswith('[korrupció klasszifikáció]'))

Filter:   0%|          | 0/24000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2250 [00:00<?, ? examples/s]

In [5]:
pos_tasks = combined_tasks.filter(lambda data: data['output'] == ' korrupció\n')
neg_tasks = combined_tasks.filter(lambda data: data['output'] != ' korrupció\n')

Filter:   0%|          | 0/24000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2250 [00:00<?, ? examples/s]

Filter:   0%|          | 0/24000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2250 [00:00<?, ? examples/s]

In [6]:
neg_tasks

DatasetDict({
    train: Dataset({
        features: ['theme', 'input', 'output'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['theme', 'input', 'output'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['theme', 'input', 'output'],
        num_rows: 750
    })
})

In [7]:
pos_tasks['train'] = pos_tasks['train'].select(range(250))
pos_tasks['validation'] = pos_tasks['validation'].select(range(150))
pos_tasks['test'] = pos_tasks['test'].select(range(75))

In [8]:
from datasets import concatenate_datasets, load_dataset, DatasetDict

real_dataset = DatasetDict()

real_dataset['train'] = concatenate_datasets([pos_tasks['train'], neg_tasks['train']])
real_dataset['validation'] = concatenate_datasets([pos_tasks['validation'], neg_tasks['validation']])
real_dataset['test'] = concatenate_datasets([pos_tasks['test'], neg_tasks['test']])

real_dataset

DatasetDict({
    train: Dataset({
        features: ['theme', 'input', 'output'],
        num_rows: 8250
    })
    validation: Dataset({
        features: ['theme', 'input', 'output'],
        num_rows: 1650
    })
    test: Dataset({
        features: ['theme', 'input', 'output'],
        num_rows: 825
    })
})

In [9]:
from tqdm import tqdm
device = "cuda:0"

n = 0
k = 0
i = 0

tp = 0
fp = 0
tn = 0
fn = 0

positive = 'korrupció'
negative = 'egyéb'

for task_in, task_out in tqdm(zip(real_dataset['test']['input'], real_dataset['test']['output']), total=len(real_dataset['test'])):
    if not task_in.startswith('[korrupció klasszifikáció]'):
        continue
    with torch.cuda.amp.autocast():
        text = task_in
        inputs = tokenizer(text, return_tensors="pt").to(device)
        if len(inputs['input_ids'][0]) > 2048:
            continue
        outputs = model.generate(**inputs, max_new_tokens=4)
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        #print(result[result.rfind(':')+2:].strip())
        #print(task_out.strip())
        n += 1
        sresult = result[result.rfind(':')+2:].strip()
        slabel = task_out.strip()
        if sresult not in ['korrupció', 'egyéb']:
            i += 1
        if sresult == positive and slabel == positive:
            tp += 1
        elif sresult == negative and slabel == negative:
            tn += 1
        elif sresult == positive and slabel == negative:
            fp += 1
        elif sresult == negative and slabel == positive:
            fn += 1
        if task_out.strip() == (result[result.rfind(':')+2:].strip()):
            k += 1
    if n % 10 == 0 and n > 0:
        print(k/n, k, n, i)
        print('precision', tp/(tp+fp))
        print('recall', tp/(tp+fn))
        print('accuracy', (tp+tn)/(tp+tn+fp+fn))
        print(tp, tn, fp, fn)


print(k/n, k, n, i)

print(tp, tn, fp, fn)


  1%|█▌                                                                                                                            | 10/825 [00:07<09:01,  1.51it/s]

0.9 9 10 0
precision 1.0
recall 0.9
accuracy 0.9
9 0 0 1


  2%|███                                                                                                                           | 20/825 [00:14<08:49,  1.52it/s]

0.95 19 20 0
precision 1.0
recall 0.95
accuracy 0.95
19 0 0 1


  4%|████▌                                                                                                                         | 30/825 [00:20<08:47,  1.51it/s]

...

0.9308641975308642 754 810 0
precision 0.5772357723577236
recall 0.9466666666666667
accuracy 0.9308641975308642
71 683 52 4


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 820/825 [08:17<00:02,  1.71it/s]

0.9317073170731708 764 820 0
precision 0.5772357723577236
recall 0.9466666666666667
accuracy 0.9317073170731708
71 693 52 4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 825/825 [08:20<00:00,  1.65it/s]

0.9321212121212121 769 825 0
71 698 52 4


In [10]:
print('precision', tp/(tp+fp))
print('recall', tp/(tp+fn))
print('accuracy', (tp+tn)/(tp+tn+fp+fn))


precision 0.5772357723577236
recall 0.9466666666666667
accuracy 0.9321212121212121
